In [ ]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-2.4.6'
spark_version = 'spark-2.4.7'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [40.7 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:11 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:13 https://developer.download.nvi

In [ ]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2020-12-16 03:40:20--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  1.02MB/s    in 0.9s    

2020-12-16 03:40:22 (1.02 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [ ]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url="https://data.edd.ca.gov/resource/bcij-5wym.csv"
spark.sparkContext.addFile(url)
unemployment_rate_by_age = spark.read.csv(SparkFiles.get("bcij-5wym.csv"), sep=",", header=True, inferSchema=True)
unemployment_rate_by_age = unemployment_rate_by_age[["area_name","year","age_16_19","age_20_24","age_25_34","age_35_44","age_45_54","age_55_64","age_65"]]
# Show DataFrame
unemployment_rate_by_age.show()

+----------+----+---------+---------+---------+---------+---------+---------+------+
| area_name|year|age_16_19|age_20_24|age_25_34|age_35_44|age_45_54|age_55_64|age_65|
+----------+----+---------+---------+---------+---------+---------+---------+------+
|California|2020|    0.225|    0.152|    0.099|    0.076|    0.076|     0.08| 0.091|
|California|2020|    0.221|    0.146|    0.095|    0.071|    0.072|    0.075| 0.088|
|California|2020|    0.215|    0.138|    0.087|    0.066|    0.068|    0.069| 0.081|
|California|2020|    0.211|    0.129|    0.079|    0.059|    0.063|    0.063| 0.073|
|California|2020|      0.2|    0.118|    0.071|    0.052|    0.057|    0.058| 0.066|
|California|2020|    0.186|    0.105|    0.062|    0.045|     0.05|     0.05| 0.058|
|California|2020|    0.167|    0.091|    0.051|    0.038|    0.041|    0.043| 0.046|
|California|2020|    0.151|    0.076|    0.041|    0.029|    0.031|    0.034| 0.036|
|California|2020|    0.148|    0.073|    0.041|    0.029|    0.03

In [ ]:
url="https://data.edd.ca.gov/resource/e6gw-gvii.csv"
spark.sparkContext.addFile(url)
local_area_unemployment_stats = spark.read.csv(SparkFiles.get("e6gw-gvii.csv"), sep=",", header=True, inferSchema=True)
local_area_unemployment_stats = local_area_unemployment_stats[["area_name","year","employment","unemployment","unemployment_rate"]]
# Show DataFrame
local_area_unemployment_stats.show()

+-------------------+----+----------+------------+-----------------+
|          area_name|year|employment|unemployment|unemployment_rate|
+-------------------+----+----------+------------+-----------------+
|     Alameda County|2020|    765400|       65500|            0.079|
|      Alpine County|2020|       420|          50|            0.097|
|      Amador County|2020|     13490|        1130|            0.077|
|       Butte County|2020|     89100|        7100|            0.074|
|   Calaveras County|2020|     19370|        1370|            0.066|
|      Colusa County|2020|      9080|         930|            0.093|
|Contra Costa County|2020|    508900|       43700|            0.079|
|   Del Norte County|2020|      8670|         750|             0.08|
|   El Dorado County|2020|     86000|        5900|            0.064|
|      Fresno County|2020|    408700|       41700|            0.093|
|       Glenn County|2020|     12370|         830|            0.063|
|    Humboldt County|2020|     574

In [ ]:
url="https://data.edd.ca.gov/resource/4yzm-uyfq.csv"
spark.sparkContext.addFile(url)
longterm_occupational_employment = spark.read.csv(SparkFiles.get("4yzm-uyfq.csv"), sep=",", header=True, inferSchema=True)
longterm_occupational_employment = longterm_occupational_employment[["area_name","period","occupational_title","percentage_change","median_hourly_wage","median_annual_wage","entry_level_education"]]
# Show DataFrame
longterm_occupational_employment.show()

+--------------------+---------+--------------------+-----------------+------------------+------------------+---------------------+
|           area_name|   period|  occupational_title|percentage_change|median_hourly_wage|median_annual_wage|entry_level_education|
+--------------------+---------+--------------------+-----------------+------------------+------------------+---------------------+
|Anaheim-Santa Ana...|2016-2026|Total, All Occupa...|              9.3|             19.83|             41246|                  N/A|
|Anaheim-Santa Ana...|2016-2026|Management Occupa...|             10.4|             58.11|            120871|                  N/A|
|Anaheim-Santa Ana...|2016-2026|      Top Executives|              9.0|               0.0|                 0|                  N/A|
|Anaheim-Santa Ana...|2016-2026|    Chief Executives|             -3.7|               0.0|                 0|    Bachelor's degree|
|Anaheim-Santa Ana...|2016-2026|General and Opera...|             10.8|     

In [ ]:
# Load in a sql function to use columns
from pyspark.sql.functions import col

url="https://data.edd.ca.gov/resource/pwxn-y2g5.csv"
spark.sparkContext.addFile(url)
occupational_employment_stat = spark.read.csv(SparkFiles.get("pwxn-y2g5.csv"), sep=",", header=True, inferSchema=True)
occupational_employment_stat = occupational_employment_stat[["area_name","year","wage_type","occupational_title","mean_wage"]]
occupational_employment_stat = occupational_employment_stat.filter(col("wage_type") == "Hourly wage")

# Show DataFrame
occupational_employment_stat.show()

+----------+----+-----------+--------------------+---------+
| area_name|year|  wage_type|  occupational_title|mean_wage|
+----------+----+-----------+--------------------+---------+
|California|2020|Hourly wage|Accountants and A...|    41.07|
|California|2020|Hourly wage|              Actors|    30.23|
|California|2020|Hourly wage|           Actuaries|    57.72|
|California|2020|Hourly wage|Acupuncturists an...|    38.18|
|California|2020|Hourly wage|Adhesive Bonding ...|    17.97|
|California|2020|Hourly wage|Administrative La...|    59.38|
|California|2020|Hourly wage|Administrative Se...|    56.31|
|California|2020|Hourly wage|Adult Basic Educa...|    42.04|
|California|2020|Hourly wage|Advertising and P...|    73.09|
|California|2020|Hourly wage|Advertising Sales...|    37.26|
|California|2020|Hourly wage|Aerospace Enginee...|    33.76|
|California|2020|Hourly wage| Aerospace Engineers|    61.38|
|California|2020|Hourly wage|Agents and Busine...|    52.87|
|California|2020|Hourly 

In [ ]:
url="https://occupation-trends-bucket.s3-us-west-1.amazonaws.com/Current_Employment_Statistics__CES_.csv"
spark.sparkContext.addFile(url)
current_employment_stats = spark.read.csv(SparkFiles.get("Current_Employment_Statistics__CES_.csv"), sep=",", header=True, inferSchema=True)
current_employment_stats = current_employment_stats[["Area Name","Year","Industry Title","Current Employment"]]
current_employment_stats = current_employment_stats[current_employment_stats["Area Name"].contains('County')]

current_employment_stats = current_employment_stats.withColumnRenamed("Area Name", "area_name") \
        .withColumnRenamed("Year", "year") \
        .withColumnRenamed("Industry Title", "industry_title") \
        .withColumnRenamed("Current Employment", "current_employment") 


# Show DataFrame
current_employment_stats.show()

+--------------+----+--------------------+------------------+
|     area_name|year|      industry_title|current_employment|
+--------------+----+--------------------+------------------+
|Alameda County|2014|Leisure and Hospi...|             61800|
|Alameda County|2014|Health Care and S...|             95300|
|Alameda County|2014|Educational and H...|            110700|
|Alameda County|2014|        Retail Trade|             68500|
|Alameda County|2014|Administrative an...|             35200|
|Alameda County|2014|         Information|             13900|
|Alameda County|2014|     Goods Producing|            100000|
|Alameda County|2014|Arts, Entertainme...|              8800|
|Alameda County|2014|    Nondurable Goods|             21100|
|Alameda County|2014|      Other Services|             24400|
|Alameda County|2014|          Total Farm|               500|
|Alameda County|2014|Natural Resources...|             34700|
|Alameda County|2014|Financial Activities|             28300|
|Alameda

Postgres Setup

In [ ]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://occupation-trends.cppwghmqrqzq.us-west-1.rds.amazonaws.com:5432/occupation-trends-db"
config = {"user":"root", 
          "password": "data1234", 
          "driver":"org.postgresql.Driver"}


In [ ]:
# Write DataFrame to active_user table in RDS

unemployment_rate_by_age.write.jdbc(url=jdbc_url, table='unemployment_rate_by_age', mode=mode, properties=config)

In [ ]:
# Write dataframe to billing_info table in RDS

local_area_unemployment_stats.write.jdbc(url=jdbc_url, table='local_area_unemployment_stats', mode=mode, properties=config)

In [ ]:
# Write dataframe to payment_info table in RDS

longterm_occupational_employment.write.jdbc(url=jdbc_url, table='longterm_occupational_employment', mode=mode, properties=config)

In [ ]:
# Write dataframe to payment_info table in RDS

occupational_employment_stat.write.jdbc(url=jdbc_url, table='occupational_employment_stats', mode=mode, properties=config)

In [ ]:
# Write dataframe to payment_info table in RDS

current_employment_stats.write.jdbc(url=jdbc_url, table='current_employment_stats', mode=mode, properties=config)